In [45]:
from fixtures import calculate_team_stats, get_next_opponents, opponents_to_dataframe, opponents_to_dataframe_def, stats_to_dataframe
from fetcher import get_teams_dict, fetch_fixtures
import pandas as pd
import numpy as np

In [46]:
# Fetch the fixture data
fixtures = fetch_fixtures()
teams = get_teams_dict()

In [47]:
CURRENT_GAMEWEEK = 6
NEXT_GAMEWEEK = 4
PREV_GAMEWEEK = 4

In [48]:
stats = calculate_team_stats(fixtures, 6, PREV_GAMEWEEK)

# Convert to DataFrame
df = stats_to_dataframe(stats, teams)

In [49]:
# Define the range and number of clusters
range_min, range_max = df['Points'].min(), df['Points'].max()
goal_range_min, goal_range_max = df['Goals Scored'].min(), df['Goals Scored'].max()
conceded_range_min, conceded_range_max = df['Goals Conceded'].min(), df['Goals Conceded'].max()
num_clusters = 5

# Generate evenly spaced data points for the clusters
points_bracket = np.linspace(range_min, range_max, num_clusters)
goals_bracket = np.linspace(goal_range_min, goal_range_max, num_clusters)
conceded_bracket = np.linspace(conceded_range_min, conceded_range_max, num_clusters)

bracket_labels = [1, 2, 3, 4]
bracket_labels_rev = [4, 3, 2, 1]

df['point_bracket'] = pd.cut(df['Points'], bins=points_bracket, labels=bracket_labels, include_lowest=True)
df['goals_bracket'] = pd.cut(df['Goals Scored'], bins=goals_bracket, labels=bracket_labels, include_lowest=True)
df['conceded_bracket'] = pd.cut(df['Goals Conceded'], bins=conceded_bracket, labels=bracket_labels_rev, include_lowest=True)
df[["Team", "point_bracket", "goals_bracket", "conceded_bracket"]]

,Team,point_bracket,goals_bracket,conceded_bracket
14,Chelsea,4,4,4
9,Aston Villa,4,4,3
19,Liverpool,4,3,4
7,Fulham,4,2,4
13,Man City,4,3,3
0,Arsenal,4,3,3
16,Newcastle,3,2,3
5,Bournemouth,3,2,2
17,Spurs,3,3,4
10,Nott'm Forest,2,1,4


In [50]:
id_point_bracket_map = df.set_index('ID')['point_bracket'].to_dict()

In [51]:
# Next GW
opponents = get_next_opponents(fixtures, CURRENT_GAMEWEEK, 1)

df_opp = opponents_to_dataframe(opponents, id_point_bracket_map, teams)
df_opp = df_opp.sort_values(by=["opp_diff", "diff"], ascending=[True, False])
opp_field = [f"next_{i}_opp" for i in range(1, 2)]
field = ["team_name"]
field = field + (opp_field)
field.append("diff")
field.append("opp_diff")

df_opp[field]

,team_name,next_1_opp,diff,opp_diff
2,Arsenal,Southampton h,4,0.5
4,Brentford,Wolves h,2,0.5
1,Liverpool,Crystal Palace a,4,1.0
14,Aston Villa,Man Utd h,4,1.0
16,Chelsea,Nott'm Forest h,4,1.0
7,Bournemouth,Leicester a,3,1.0
19,Spurs,Brighton a,3,1.0
11,Ipswich,West Ham a,2,1.0
10,West Ham,Ipswich h,1,1.0
13,Newcastle,Everton a,3,1.5


In [52]:
# Next 4 GW, Custom FDR
opponents = get_next_opponents(fixtures, CURRENT_GAMEWEEK, NEXT_GAMEWEEK)

df_opp = opponents_to_dataframe(opponents, id_point_bracket_map, teams)
df_opp = df_opp.sort_values(by=["opp_diff", "diff"], ascending=[True, False])
opp_field = [f"next_{i}_opp" for i in range(1, NEXT_GAMEWEEK+1)]
field = ["team_name"]
field = field + (opp_field)
field.append("diff")
field.append("opp_diff")

df_opp[field]

,team_name,next_1_opp,next_2_opp,next_3_opp,next_4_opp,diff,opp_diff
11,Ipswich,West Ham a,Everton h,Brentford a,Leicester h,2,1.6
19,Spurs,Brighton a,West Ham h,Crystal Palace a,Aston Villa h,3,1.8
17,Nott'm Forest,Chelsea a,Crystal Palace h,Leicester a,West Ham h,2,1.8
5,Wolves,Brentford a,Man City h,Brighton a,Crystal Palace h,1,2.0
6,Leicester,Bournemouth h,Southampton a,Nott'm Forest h,Ipswich a,1,2.0
8,Man City,Fulham h,Wolves a,Southampton h,Bournemouth a,4,2.2
4,Brentford,Wolves h,Man Utd a,Ipswich h,Fulham a,2,2.2
10,West Ham,Ipswich h,Spurs a,Man Utd h,Nott'm Forest a,1,2.2
1,Liverpool,Crystal Palace a,Chelsea h,Arsenal a,Brighton h,4,2.4
12,Everton,Newcastle h,Ipswich a,Fulham h,Southampton a,2,2.4


In [53]:
# Next 4 GW, Default FDR
df_opp = opponents_to_dataframe_def(opponents, teams)
df_opp = df_opp.sort_values(by=["opp_diff"], ascending=[True])
opp_field = [f"next_{i}_opp" for i in range(1, NEXT_GAMEWEEK+1)]
field = ["team_name"]
field = field + (opp_field)
field.append("opp_diff")

df_opp[field]

,team_name,next_1_opp,next_2_opp,next_3_opp,next_4_opp,opp_diff
11,Ipswich,West Ham a,Everton h,Brentford a,Leicester h,2.0
6,Leicester,Bournemouth h,Southampton a,Nott'm Forest h,Ipswich a,2.0
8,Man City,Fulham h,Wolves a,Southampton h,Bournemouth a,2.0
12,Everton,Newcastle h,Ipswich a,Fulham h,Southampton a,2.2
19,Spurs,Brighton a,West Ham h,Crystal Palace a,Aston Villa h,2.4
17,Nott'm Forest,Chelsea a,Crystal Palace h,Leicester a,West Ham h,2.4
4,Brentford,Wolves h,Man Utd a,Ipswich h,Fulham a,2.4
15,Man Utd,Aston Villa a,Brentford h,West Ham a,Chelsea h,2.6
10,West Ham,Ipswich h,Spurs a,Man Utd h,Nott'm Forest a,2.6
0,Crystal Palace,Liverpool h,Nott'm Forest a,Spurs h,Wolves a,2.6
